# GPT Filtering

In [ ]:
!pip install openai

In [ ]:
# Once you add your API key below, make sure to not share it with anyone! The API key should remain private.
from openai import OpenAI
OPENAI_API_KEY='###PRIVATE API###' 

client = OpenAI(
  api_key=OPENAI_API_KEY,
)

# Load and process data

In [ ]:
import json

with open("/kaggle/input/post-process-llm/law.json", 'r', encoding = 'utf-8') as f:
    law_data = json.load(f)
with open("/kaggle/input/post-process-llm/private_test_TASK_1.json", 'r', encoding = 'utf-8') as f:
    prv = json.load(f)
    
current_th_path = "/kaggle/input/post-process-llm/private_24_th355_reordered_file.json"
with open(current_th_path, 'r', encoding = 'utf-8') as f:
    inferred = json.load(f)
    

In [ ]:
law_dict = {}
for law in law_data:
    article_dict = {}
    for article in law["articles"]:
        article_dict[article["id"]] = article["text"]
    law_dict[law['id']] = article_dict
    
question_dict = {}
for question in prv:
    question_dict[question["question_id"]] = question

In [ ]:
for question in inferred:
    question_id = question["question_id"]
    question["query"] = question_dict[question_id]["text"]
    for article in question["relevant_articles"]:
        law_id = article["law_id"]
        article_id = article["article_id"]
        article["content"] = law_dict[law_id][article_id]

with open('gpt_postprocess.json', 'w', encoding='utf-8') as f:
    json.dump(inferred, f, ensure_ascii=False, indent=4)

# GPT prompting to label article as relevant ("1") or redundant ("0")

In [ ]:
def generate_answer(question, relevant_articles):
    responses = []
    for relevant_article in relevant_articles:
        prompt = f'''Với đầu vào là câu hỏi và điều luật liên quan, bạn hãy đánh giá liệu điều luật đó có thực sự dùng để trả lời cho câu hỏi hay không? 
        Câu hỏi: "{question}"
        Điều luật liên quan: "{relevant_article["content"]}"
        
        Bạn chỉ được phép trả lời 1 nếu điều luật này thực sự được dùng để trả lời câu hỏi, hoặc 0 nếu điều luật đó không được dùng để trả lời câu hỏi, và bạn không được phép trả lời thêm bất cứ điều gì khác.'''
        
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        response = str(completion.choices[0].message.content)
        responses.append(response)
    return responses


In [ ]:
for question in inferred:
    if len(question["relevant_articles"]) > 1:
        responses = generate_answer(question["query"], question["relevant_articles"])
        question["llm_responses"] = responses
        
with open('llm_responses_th355.json', 'w', encoding='utf-8') as f:
    json.dump(inferred, f, ensure_ascii=False, indent=4)

# GPT Matching - Remove article responded "0"

In [31]:
import json
path = "/kaggle/input/llm-matching/updated_llm_responses_th2_5.json"
with open(path, 'r', encoding = 'utf-8') as f:
    inferred = json.load(f)

In [32]:
for question in inferred:
    if "llm_responses" in question:
        responses = question["llm_responses"]
        if "1" in responses:
            for i in range(len(responses)):
                if responses[i] == "0":
                    question["relevant_articles"].pop(i)
                    i -= 1
        del question["llm_responses"]

In [33]:
with open('postprocessed_gpt.json', 'w', encoding='utf-8') as f:
    json.dump(inferred, f, ensure_ascii=False, indent=4)